In [2]:
pip install gym[toy_text]

     ---------------------------------------- 4.8/4.8 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.2
    Uninstalling pygame-2.5.2:
      Successfully uninstalled pygame-2.5.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Toegang geweigerd: 'C:\\Users\\Admin\\anaconda3\\Lib\\site-packages\\~ygame\\base.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import gym

print(gym.__version__)

"""
You will not be able to see the simulation in colab. You can do it
locally by un-commenting the following 4 lines and env.render() in the
last piece of the code
"""
import pygame
pygame.init()
pygame.display.init()
pygame.display.list_modes()


# We will load a DiscreteEnv and retrieve the probability and reward
# information
env = gym.make("FrozenLake8x8-v1", desc=None, map_name=None, is_slippery=True)
"""
DiscreteEnv has an attribute P which holds everything we want as a
dictionary of lists:
P[s][a] = [(probability, nextstate, reward, done), ...]
"""


# first, we initialize the structures with zeros
prob = {i: {j: {a: 0 for a in range(env.action_space.n)}
            for j in range(env.observation_space.n)}
        for i in range(env.observation_space.n)}
rewd = {i: {j: {a: 0 for a in range(env.action_space.n)}
            for j in range(env.observation_space.n)}
        for i in range(env.observation_space.n)}
# then, we fill them with the actual information
for i in range(env.observation_space.n):
    for a in range(env.action_space.n):
        for (p, j, r, d) in env.P[i][a]:
            prob[i][j][a] += p
            rewd[i][j][a] += r


# Policy computation: here's where YOU code
"""
Insert your clever policy computation here! make sure to replace the
policy dictionary below by the results of your computation
"""
T = 10  # Given horizon
policy = {t: {i: env.action_space.sample()
              for i in range(env.observation_space.n)}
          for t in range(T)}



# Policy evaluation: here's where YOU also code
"""
Insert here your code to evaluate
the total expected rewards over the planning horizon T
if one follows your policy. Do the same for a random policy (i.e. the
sample policy given above). As a sanity check, your policy should get an
expected reward of at least the one obtained by the random policy!
"""



# # Simulation: you can try your policy here
# state = env.reset()
# for t in range(T):
#     #env.render()
#     action = policy[t][state]
#     print(state)
#     print(f"Action = {action}")
#     state, reward, done, _ = env.step(action)
#     # if the MDP is stuck, we end the simulation here
#     if done:
#         print(state)
#         print(f"Episode finished after {t + 1} timesteps")
#         break
# env.close()


# Simulation: you can try your policy here
state = env.reset()
for i, t in enumerate(range(T)):
    env.render()
    if i ==0:
        action = policy[t][state[0]]
        print(f"Action = {action}")
        state, reward, done, _, _ = env.step(action)
        # print(state, reward, done)
    else:
        action = policy[t][state]
        print(f"Action = {action}")
        state, reward, done, _, _ = env.step(action)
        # print(state, reward, done)

    # if the MDP is stuck, we end the simulation here
    if done:
        print(f"Episode finished after {t + 1} timesteps")
        break
env.close()

0.26.2
Action = 2
Action = 0
Action = 2
Action = 2
Action = 2
Action = 0
Action = 2
Action = 0
Action = 1
Action = 3


It looks we are given the transition of the game.

In [25]:
state = 62
action = 1
print(env.P[state][action])
#[(probability, nextstate, reward, done), ...]

[(0.3333333333333333, 61, 0.0, False), (0.3333333333333333, 62, 0.0, False), (0.3333333333333333, 63, 1.0, True)]


What we need to compute is an approximate optimal policy in every state.
We 

In [117]:
states = range(len(env.P)) # must be immutable
v = {state:0 for state in states} 
policy = {state:0 for state in states} # assume that 0 is an action
q = {state:{action:0 for action in range(len(env.P[state])) } for state in states}

vv = []
pp = []
qq = []

for i in range(1000):
    for state in states:
        info = env.P[state][policy[state]]
        v[state] = sum(b[0]*b[2]+v[b[1]] for b in info)/len(info)

    for state in range(len(env.P)):
        for action in range(len(env.P[state])): 
            val = sum(b[0]*b[2]+v[b[1]] for b in env.P[state][action])/len(env.P[state][action])
            q[state][action]= val

    for state in states:
        ma , mval = 0, q[state][0]
        for action,val in q[state].items():
            if val>mval:
                ma = action 
                mval = val
        policy[state] = ma
    vv.append(v.copy())
    pp.append(policy.copy())
    qq.append(q.copy())

In [120]:
import ipywidgets as widgets
from ipywidgets import interactive

def plot_vv(i): 
    visv(vv[i])

i_slider = widgets.IntSlider(value=0, min=0, max=len(vv) - 1, description='i')
interactive_plot = interactive(plot_vv, i=i_slider)
interactive_plot

interactive(children=(IntSlider(value=0, description='i', max=99), Output()), _dom_classes=('widget-interact',…

In [78]:
import matplotlib.pyplot as plt
#chatgpt wrote this
def visq(q):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    # Find the minimum and maximum Q-values across all actions and states
    min_q = min(q[state].get(action, 0) for action in range(4) for state in range(64))
    max_q = max(q[state].get(action, 0) for action in range(4) for state in range(64))
    for action in range(4):
        # Convert 'q' for the specific action to a 2D numpy array
        q_array = np.zeros((8, 8))
        for state in range(64):
            row, col = divmod(state, 8)
            q_value = q[state].get(action, 0)
            q_array[row][col] = q_value
        # Create a heatmap for this action with a common color scale
        ax = axes[action // 2, action % 2]
        im = ax.imshow(q_array, cmap='hot', interpolation='nearest', vmin=min_q, vmax=max_q)
        ax.set_title(f'Action {action}')
        plt.colorbar(im, ax=ax)

    plt.tight_layout()
    plt.show()


#chatgpt wrote this
def visv(v):
    # Convert 'v' to a 2D numpy array
    v_array = np.zeros((8, 8))
    for state in v:
        row, col = divmod(state, 8)
        v_array[row][col] = v[state]

    # plt.imshow(v_array, cmap='hot', interpolation='nearest')
    plt.imshow(v_array, cmap='hot')
    plt.colorbar()
    plt.show()

#chatgpt wrote this
def visp(policy):
    # Convert 'v' to a 2D numpy array
    # Convert 'policy' to a 2D numpy array
    policy_array = np.zeros((8, 8))
    for state in policy:
        row, col = divmod(state, 8)
        policy_array[row][col] = policy[state]

    # Create a heatmap for the policy
    plt.imshow(policy_array, cmap='tab10', interpolation='nearest', vmin=0, vmax=3)  # Assuming 0 to 3 are the action values
    plt.colorbar(ticks=[0, 1, 2, 3])  # Set ticks to represent the action values

    # Show the plot
    plt.show()